In [6]:
import numpy as np
import os
import pandas as pd

In [7]:
saveState = False
patNumbers = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
pd.set_option("display.precision", 3, "styler.format.decimal", ",")

medianErrorsStages = np.zeros(shape=(12, 5)) # shape=(patients, reg_stages)
runTime = np.zeros(shape=(12, 3))
lastIterMetrics = np.zeros(shape=(12, 3))

for idx, num in enumerate(patNumbers):
    errorPath = f"input_files\\pacient_{num}\\registration\\"

    preFile = np.load(f"{errorPath}gradient\\pacient{num}PreErrors.npz")
    initialFile = np.load(f"{errorPath}gradient\\pacient{num}InitialErrors.npz")
    gradFile = np.load(f"{errorPath}gradient\\pacient{num}RegErrors.npz")
    gradLineFile = np.load(f"{errorPath}gradientline\\pacient{num}RegErrors.npz")
    gradLbfFile = np.load(f"{errorPath}gradientlbf\\pacient{num}RegErrors.npz")
    
    gradRegFile = np.load(f"{errorPath}gradient\\pacient{num}RegParams.npz")
    gradLineRegFile = np.load(f"{errorPath}gradientline\\pacient{num}RegParams.npz")
    gradLbfRegFile = np.load(f"{errorPath}gradientlbf\\pacient{num}RegParams.npz")

    # median errors per patient and registration stage
    medianErrorsStages[idx, 0] = preFile["preMedians"] # before initialization
    medianErrorsStages[idx, 1] = initialFile["initialMedians"] # after initialization
    medianErrorsStages[idx, 2] = gradFile["regMedians"][-1] # gradient 
    medianErrorsStages[idx, 3] = gradLineFile["regMedians"][-1] # gradient line 
    medianErrorsStages[idx, 4] = gradLbfFile["regMedians"][-1] # lbfgs
    
    lastIterMetrics[idx, 0] = gradRegFile["metricValues"][-1]
    lastIterMetrics[idx, 1] = gradLineRegFile["metricValues"][-1]
    lastIterMetrics[idx, 2] = gradLbfRegFile["metricValues"][-1]
    
    runTime[idx, 0] = gradRegFile["execTime"]
    runTime[idx, 1] = gradLineRegFile["execTime"]
    runTime[idx, 2] = gradLbfRegFile["execTime"]
    
    del(preFile, initialFile, gradFile, gradLineFile, gradLbfFile, gradRegFile,
        gradLineRegFile, gradLbfRegFile)
    

regStages = ["Před inicializací", "Po inicializaci", "Gradientní\nsestup", 
             "Gradientní sestup\nse zl. řezem", "BFGS s\nomezenou pamětí"]
regCols = ["pre", "initial", "grad", 
             "grad_line", "grad_lbf"]
regLabels = ["Před inicializací", "Po inicializaci", "Gradientní sestup", 
             "Gradientní sestup se zl. řezem", "BFGS s omezenou pamětí"]

In [8]:
initialLastmedianErrorsDf = pd.DataFrame(medianErrorsStages, columns=pd.Index(regLabels),
                                       index=pd.Index(range(1, medianErrorsStages.shape[0] + 1), name="Číslo pacienta"))
print(initialLastmedianErrorsDf.T.style.to_latex())
initialLastmedianErrorsDf.T

\begin{tabular}{lrrrrrrrrrrrr}
Číslo pacienta & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 \\
Před inicializací & 95,204770 & 99,369106 & 59,651445 & 77,284026 & 84,555336 & 125,291004 & 51,334492 & 25,338598 & 117,979494 & 30,762199 & 44,756299 & 115,841176 \\
Po inicializaci & 102,126586 & 79,781340 & 23,645868 & 83,098711 & 20,322598 & 60,211795 & 33,765366 & 14,480076 & 62,515038 & 26,047366 & 32,823615 & 42,758684 \\
Gradientní sestup & 12,518687 & 22,750668 & 6,505861 & 11,694710 & 10,801063 & 8,984546 & 6,824955 & 4,891915 & 4,121163 & 15,855933 & 18,469326 & 4,425060 \\
Gradientní sestup se zl. řezem & 12,818835 & 22,735113 & 5,527218 & 11,947224 & 11,107896 & 9,397009 & 6,148300 & 6,140963 & 3,658141 & 14,893941 & 19,661171 & 4,132670 \\
BFGS s omezenou pamětí & 10,614324 & 22,112244 & 5,308194 & 12,163014 & 11,173012 & 7,101618 & 5,968503 & 6,032467 & 3,596933 & 16,290659 & 20,742189 & 4,345826 \\
\end{tabular}


Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Před inicializací,95.205,99.369,59.651,77.284,84.555,125.291,51.334,25.339,117.979,30.762,44.756,115.841
Po inicializaci,102.127,79.781,23.646,83.099,20.323,60.212,33.765,14.480,62.515,26.047,32.824,42.759
Gradientní sestup,12.519,22.751,6.506,11.695,10.801,8.985,6.825,4.892,4.121,15.856,18.469,4.425
Gradientní sestup se zl. řezem,12.819,22.735,5.527,11.947,11.108,9.397,6.148,6.141,3.658,14.894,19.661,4.133
BFGS s omezenou pamětí,10.614,22.112,5.308,12.163,11.173,7.102,5.969,6.032,3.597,16.291,20.742,4.346


In [9]:
initialLastMetricsDf = pd.DataFrame(lastIterMetrics, columns=pd.Index(regLabels[2:]),
                                    index=pd.Index(range(1, lastIterMetrics.shape[0] + 1), name="Číslo pacienta"))
print(initialLastMetricsDf.T.style.to_latex())
initialLastMetricsDf.T

\begin{tabular}{lrrrrrrrrrrrr}
Číslo pacienta & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 \\
Gradientní sestup & -0,642979 & -0,456110 & -0,654532 & -0,545208 & -0,622273 & -0,642077 & -0,668522 & -0,625254 & -0,693658 & -0,625783 & -0,619464 & -0,662384 \\
Gradientní sestup se zl. řezem & -0,643686 & -0,456174 & -0,663128 & -0,555299 & -0,628668 & -0,644896 & -0,675744 & -0,632153 & -0,703064 & -0,632315 & -0,624210 & -0,670259 \\
BFGS s omezenou pamětí & -0,642476 & -0,457349 & -0,663696 & -0,596203 & -0,629390 & -0,639465 & -0,675661 & -0,632411 & -0,703621 & -0,634474 & -0,628881 & -0,670857 \\
\end{tabular}


Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Gradientní sestup,-0.643,-0.456,-0.655,-0.545,-0.622,-0.642,-0.669,-0.625,-0.694,-0.626,-0.619,-0.662
Gradientní sestup se zl. řezem,-0.644,-0.456,-0.663,-0.555,-0.629,-0.645,-0.676,-0.632,-0.703,-0.632,-0.624,-0.670
BFGS s omezenou pamětí,-0.642,-0.457,-0.664,-0.596,-0.629,-0.639,-0.676,-0.632,-0.704,-0.634,-0.629,-0.671


In [10]:
runTimeDf = pd.DataFrame(runTime, columns=pd.Index(regLabels[2:]), 
                         index=pd.Index(range(1, runTime.shape[0] + 1), name="Číslo pacienta"))
print(runTimeDf.T.style.to_latex())
runTimeDf.T

\begin{tabular}{lrrrrrrrrrrrr}
Číslo pacienta & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 \\
Gradientní sestup & 1,528435 & 2,108209 & 1,139139 & 1,635712 & 1,042700 & 1,205097 & 0,888419 & 0,794628 & 1,119246 & 1,018626 & 1,464743 & 1,012578 \\
Gradientní sestup se zl. řezem & 3,237116 & 4,081288 & 4,218160 & 5,560802 & 3,810409 & 5,533082 & 1,961808 & 2,310031 & 1,594754 & 4,320487 & 7,273673 & 2,060762 \\
BFGS s omezenou pamětí & 7,397465 & 7,425741 & 6,855642 & 7,320143 & 7,488316 & 5,513113 & 6,358583 & 6,385045 & 4,361580 & 6,437549 & 5,153826 & 4,784387 \\
\end{tabular}


Číslo pacienta,1,2,3,4,5,6,7,8,9,10,11,12
Gradientní sestup,1.528,2.108,1.139,1.636,1.043,1.205,0.888,0.795,1.119,1.019,1.465,1.013
Gradientní sestup se zl. řezem,3.237,4.081,4.218,5.561,3.810,5.533,1.962,2.310,1.595,4.320,7.274,2.061
BFGS s omezenou pamětí,7.397,7.426,6.856,7.320,7.488,5.513,6.359,6.385,4.362,6.438,5.154,4.784


In [12]:
with pd.ExcelWriter("graficke_vystupy\\vystupy_registrace.xlsx") as excel_writer:
    initialLastmedianErrorsDf.to_excel(excel_writer, sheet_name="median_mezibodove_vzdalenosti",
                                       float_format="%.3f",
                                       columns=initialLastmedianErrorsDf.columns)
    
    runTimeDf.to_excel(excel_writer, sheet_name="vypocetni_doba",
                       float_format="%.3f", 
                       columns=runTimeDf.columns)
    
    initialLastMetricsDf.to_excel(excel_writer, sheet_name="vzajemna_informace",
                                  float_format="%.3f",
                                  columns=initialLastMetricsDf.columns)